<a href="https://colab.research.google.com/github/Sypodko/python/blob/main/Botik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot nest_asyncio requests > /dev/null

import asyncio
import nest_asyncio
import logging
import requests
from telegram import Update, ReplyKeyboardMarkup, KeyboardButton
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
from telegram.error import BadRequest, RetryAfter
from IPython.display import display, HTML
from concurrent.futures import ThreadPoolExecutor

# Разрешаем вложенные event loops
nest_asyncio.apply()

# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Конфигурация
TELEGRAM_TOKEN = "7973664515:AAFsZfmj08sVfXK5qdn47sxdBqcBkWTq4Do"
OPENROUTER_API_KEY = "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"

# Доступные модели (бесплатные)
MODELS = {
    "Gemini Flash 🚀": "google/gemini-2.0-flash-thinking-exp:free",
    "DeepSeek V3 🧠": "deepseek/deepseek-chat-v3-0324:free",
    "Gemma 3 🌸": "google/gemma-3-27b-it:free"
}

class StableAIBot:
    def __init__(self):
        self.application = None
        self.running = False
        self.current_model = MODELS["Gemini Flash 🚀"]
        self.user_status_messages = {}
        self.executor = ThreadPoolExecutor(max_workers=10)
        self.request_queue = asyncio.Queue()
        self.rate_limit = asyncio.Semaphore(5)
        self.shutdown_event = asyncio.Event()

    def get_main_keyboard(self):
        return ReplyKeyboardMarkup(
            [
                ["✨ Задать вопрос"],
                ["🔄 Выбрать модель", "❓ О моделях"],
                ["💡 Примеры запросов"]
            ],
            resize_keyboard=True
        )

    def get_models_keyboard(self):
        buttons = []
        for name in MODELS:
            prefix = "✅ " if MODELS[name] == self.current_model else "🔹 "
            buttons.append([KeyboardButton(f"{prefix}{name}")])
        buttons.append([KeyboardButton("🏠 На главную")])
        return ReplyKeyboardMarkup(buttons, resize_keyboard=True)

    async def start_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """Обработчик команды /start"""
        try:
            welcome_msg = f"""
🌟 <b>Добро пожаловать в AI-Бота!</b> 🌟

Текущая модель: <b>{next(k for k, v in MODELS.items() if v == self.current_model)}</b>
{self.get_model_description(next(k for k, v in MODELS.items() if v == self.current_model))}

Вы можете:
1. Выбрать другую модель (кнопка 🔄 Выбрать модель)
2. Сразу задать вопрос текущей модели (кнопка ✨ Задать вопрос)
            """
            await update.message.reply_text(
                welcome_msg,
                parse_mode='HTML',
                reply_markup=self.get_main_keyboard()
            )
        except Exception as e:
            logger.error(f"Ошибка в start_command: {str(e)}")
            await update.message.reply_text(
                "Привет! Используйте кнопки для управления ботом.",
                reply_markup=self.get_main_keyboard()
            )

    async def handle_message(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        try:
            user_message = update.message.text
            user_id = update.effective_user.id

            if "❓ О моделях" in user_message:
                await self.show_models_info(update)
                return

            if "🔄 Выбрать модель" in user_message:
                await self.show_model_selection(update)
                return

            if "🏠 На главную" in user_message:
                await self.start_command(update, context)
                return

            if any(model in user_message for model in MODELS):
                selected_model = user_message.replace("✅ ", "").replace("🔹 ", "")
                if selected_model in MODELS:
                    self.current_model = MODELS[selected_model]
                    await update.message.reply_text(
                        f"🎉 Выбрана модель: <b>{selected_model}</b>\n\n"
                        f"{self.get_model_description(selected_model)}",
                        parse_mode='HTML',
                        reply_markup=self.get_main_keyboard()
                    )
                return

            if "💡 Примеры запросов" in user_message:
                await self.show_examples(update)
                return

            if "✨ Задать вопрос" in user_message:
                await update.message.reply_text(
                    f"📩 Отправьте ваш вопрос текущей модели ({next(k for k, v in MODELS.items() if v == self.current_model)})",
                    reply_markup=self.get_main_keyboard()
                )
                return

            await self.request_queue.put((update, user_message, user_id))
            await self.process_queue()

        except Exception as e:
            logger.error(f"Ошибка обработки сообщения: {str(e)}")
            await update.message.reply_text(
                "😢 Произошла ошибка, попробуйте еще раз",
                reply_markup=self.get_main_keyboard()
            )

    async def process_queue(self):
        """Обработка очереди запросов"""
        while not self.request_queue.empty():
            update, message, user_id = await self.request_queue.get()
            try:
                async with self.rate_limit:
                    await self.process_ai_request(update, message, user_id)
            except RetryAfter as e:
                await asyncio.sleep(e.retry_after)
                await self.request_queue.put((update, message, user_id))
            except Exception as e:
                logger.error(f"Ошибка обработки запроса из очереди: {str(e)}")
                await update.message.reply_text(
                    "😢 Произошла ошибка, попробуйте позже",
                    reply_markup=self.get_main_keyboard()
                )

    def get_model_description(self, model_name):
        descriptions = {
            "Gemini Flash 🚀": "⚡ <i>Быстрая как молния, идеальна для простых запросов</i>",
            "DeepSeek V3 �": "🔍 <i>Глубокий анализ, справляется со сложными задачами</i>",
            "Gemma 3 🌸": "🌿 <i>Легкая и креативная, хороша для генерации идей</i>"
        }
        return descriptions.get(model_name, "")

    async def show_models_info(self, update: Update):
        model_info = "🤖 <b>Доступные модели:</b>\n\n"
        for name in MODELS:
            model_info += f"• <b>{name}</b>\n{self.get_model_description(name)}\n\n"

        await update.message.reply_text(
            model_info,
            parse_mode='HTML',
            reply_markup=self.get_main_keyboard()
        )

    async def process_ai_request(self, update: Update, message: str, user_id: int):
        chat_id = update.effective_chat.id

        try:
            status_msg = await update.message.reply_text(
                "🌀 <i>Генерирую ответ...</i>",
                parse_mode='HTML'
            )
            self.user_status_messages[user_id] = status_msg.message_id

            loop = asyncio.get_event_loop()
            response = await loop.run_in_executor(
                self.executor,
                self.make_api_request,
                message
            )

            try:
                await update.get_bot().delete_message(
                    chat_id=chat_id,
                    message_id=self.user_status_messages[user_id]
                )
            except Exception as e:
                logger.error(f"Ошибка удаления сообщения: {str(e)}")
            finally:
                if user_id in self.user_status_messages:
                    del self.user_status_messages[user_id]

            if response.status_code == 200:
                answer = response.json()['choices'][0]['message']['content']
                for i in range(0, len(answer), 4000):
                    await update.message.reply_text(
                        answer[i:i+4000],
                        reply_markup=self.get_main_keyboard()
                    )
            else:
                error = response.json().get('error', {})
                await update.message.reply_text(
                    f"⚠️ Ошибка {response.status_code}\n"
                    f"{error.get('message', 'Попробуйте позже')}",
                    reply_markup=self.get_main_keyboard()
                )

        except Exception as e:
            logger.error(f"Ошибка обработки запроса: {str(e)}")
            try:
                if user_id in self.user_status_messages:
                    await update.get_bot().delete_message(
                        chat_id=chat_id,
                        message_id=self.user_status_messages[user_id]
                    )
            except:
                pass
            finally:
                if user_id in self.user_status_messages:
                    del self.user_status_messages[user_id]

            await update.message.reply_text(
                "😢 Произошла ошибка, попробуйте позже",
                reply_markup=self.get_main_keyboard()
            )

    def make_api_request(self, message):
        """Синхронный метод для выполнения API запроса"""
        headers = {
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://github.com",
            "X-Title": "AI Bot"
        }

        payload = {
            "model": self.current_model,
            "messages": [{"role": "user", "content": message}],
            "temperature": 0.7,
            "max_tokens": 4000
        }

        return requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=60
        )

    async def show_model_selection(self, update: Update):
        await update.message.reply_text(
            "<b>Выберите модель:</b> (текущая отмечена ✅)",
            parse_mode='HTML',
            reply_markup=self.get_models_keyboard()
        )

    async def show_examples(self, update: Update):
        examples = [
            "🌟 Напиши стихотворение о космосе",
            "📊 Объясни квантовую физику просто",
            "💡 Придумай 5 идей для стартапа",
            "👨‍💻 Помоги с кодом на Python"
        ]
        await update.message.reply_text(
            "🎯 <b>Примеры запросов:</b>\n\n" + "\n".join(examples),
            parse_mode='HTML',
            reply_markup=self.get_main_keyboard()
        )

    async def cleanup(self):
        """Корректное завершение всех ресурсов"""
        try:
            if self.application:
                if hasattr(self.application, 'updater') and self.application.updater.running:
                    await self.application.updater.stop()
                await self.application.stop()
                await self.application.shutdown()

            while not self.request_queue.empty():
                self.request_queue.get_nowait()

            self.executor.shutdown(wait=True)

        except Exception as e:
            logger.error(f"Ошибка при завершении работы: {str(e)}")

    async def run_bot(self):
        """Основной метод запуска бота"""
        try:
            self.application = Application.builder() \
                .token(TELEGRAM_TOKEN) \
                .pool_timeout(30) \
                .read_timeout(30) \
                .write_timeout(30) \
                .get_updates_read_timeout(30) \
                .build()

            self.application.add_handler(CommandHandler("start", self.start_command))
            self.application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, self.handle_message))

            self.application.add_error_handler(self.error_handler)

            display(HTML("""
            <style>
                .bot-status {
                    background: linear-gradient(135deg, #6e48aa, #9d50bb);
                    color: white;
                    padding: 15px;
                    border-radius: 10px;
                    font-family: Arial;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.2);
                }
            </style>
            <div class="bot-status">
                <h3>🎉 AI Bot запущен!</h3>
                <p>Отправьте /start в Telegram</p>
            </div>
            """))

            await self.application.initialize()
            await self.application.start()
            await self.application.updater.start_polling()

            self.running = True
            while self.running and not self.shutdown_event.is_set():
                await asyncio.sleep(1)

        except Exception as e:
            display(HTML(f"""
            <div style="
                background: #ff4d4d;
                color: white;
                padding: 15px;
                border-radius: 5px;
            ">
                <h3>😢 Ошибка запуска</h3>
                <p>{str(e)}</p>
            </div>
            """))
            logger.error(f"Ошибка в основном цикле: {str(e)}")
        finally:
            await self.cleanup()

    async def error_handler(self, update: object, context: ContextTypes.DEFAULT_TYPE):
        logger.error(f"Ошибка: {context.error}", exc_info=context.error)

def main():
    bot = StableAIBot()
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    try:
        # Запускаем основную корутину
        main_task = loop.create_task(bot.run_bot())

        # Обработка Ctrl+C
        def shutdown():
            bot.shutdown_event.set()
            bot.running = False
            if not main_task.done():
                main_task.cancel()

        try:
            import signal
            for sig in (signal.SIGINT, signal.SIGTERM):
                loop.add_signal_handler(sig, shutdown)
        except (NotImplementedError, RuntimeError):
            pass

        loop.run_until_complete(main_task)

    except KeyboardInterrupt:
        logger.info("Получен сигнал завершения, останавливаем бота...")
    except Exception as e:
        logger.error(f"Неожиданная ошибка: {str(e)}")
    finally:
        tasks = asyncio.all_tasks(loop=loop)
        for task in tasks:
            task.cancel()

        loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))
        loop.close()
        logger.info("Бот завершил работу")

if __name__ == '__main__':
    main()